In [8]:
!pip install flatbuffers
!pip install pypng

    100% |████████████████████████████████| 655kB 2.0MB/s 
  Stored in directory: /Users/Mazen/Library/Caches/pip/wheels/41/6b/ef/0493b536b6d4722c2ae9486691b1d49b922b9877922beeabb3
Successfully built pypng


In [3]:
import flatbuffer_reader as reader
import subprocess
import numpy as np
import math
import re
import pandas as pd
import os
import csv

In [4]:
class Utils:

  def format_bytes(self, bytes):

    if bytes < 1024:
      return str(bytes) + " Bytes"
    elif bytes < (1024*1024):
      return '{0:1.3g}'.format(bytes/1024) + " Kilobytes"
    elif bytes < (1024*1024*1024):
      return '{0:1.3g}'.format(bytes/(1024*1024)) + " Megabytes"
    elif bytes < (1024*1024*1024*1024):
      return '{0:1.3g}'.format(bytes/(1024*1024*1024)) + " Gigabytes"


  def format_left(self, str, width, pad_char = " "):

    if len(str) >= width:
        return str

    right_pad = width - len(str)

    return str + (pad_char * right_pad)

  def format_centre(self, str, width, pad_char = " "):

    if len(str) >= width:
        return str

    left_pad = math.floor((width - len(str)) / 2)
    right_pad = width - len(str) - left_pad
    
    #return (pad_char * left_pad) + str + (pad_char * right_pad)
    return (pad_char * int(left_pad)) + str + (pad_char * int(right_pad))

  def print_weights_summary(self, model):

    print("\nWeights Summary:")
    print("\n%s Weight tensors, containing %s weights, taking %s\n" %
          ('{:,}'.format(model.weights_tensor_count),
           '{:,}'.format(model.total_weights),
           self.format_bytes(model.total_weights_bytes)))
    for weight_type in model.weight_types:
        totals = model.weight_types[weight_type]
        if totals['weights'] > 0:
            print("  %15s %10s weights taking %s" %
                  ('{:,}'.format(totals['weights']),
                   model.types[weight_type],
                   self.format_bytes(totals['bytes'])))

    max_name_len = 0
    max_size_len = 0
    for i, tensor in enumerate(model.tensors):
        if tensor.Buffer() != 0 and model.buffers[tensor.Buffer()].DataLength() != 0:
            max_name_len = max(max_name_len, len(tensor.Name().decode("utf-8")))
        shape_str = "scalar"

        tensors_array = tensor.ShapeAsNumpy()
        if type(tensors_array) is int:   # is not array (only 1 element)
          tensors_array = np.array([tensors_array])
        
        if tensors_array.size > 0:
            dim_strings = []
            for d in tensors_array:
                dim_strings += [str(d)]
            
            shape_str = "(%s)" % (', '.join(dim_strings))
        
        max_size_len = max(max_size_len, len(shape_str))

    header_string = "Weights Details%s   type     %s      size      " % \
                          ((" " * (max_name_len - 13)),
                          self.format_centre(" (shape) ", max_size_len, "-"))
    print("\n%s\n%s" %
          (header_string,
          "-" * len(header_string)))
    
    for i, tensor in enumerate(model.tensors):
        # if this tensor isn't pointing to the null buffer and is defined
        if tensor.Buffer() != 0 and model.buffers[tensor.Buffer()].DataLength() != 0:
            name = tensor.Name().decode("utf-8")
            shape_str = "scalar"

            tensors_array = tensor.ShapeAsNumpy()
            if type(tensors_array) is int:   # is not array (only 1 element)
              tensors_array = np.array([tensors_array])

            if tensors_array.size > 0:
                dim_strings = []
                for d in tensors_array:
                    dim_strings += [str(d)]
                shape_str = "(%s)" % (', '.join(dim_strings))

            size_bytes = model.buffers[tensor.Buffer()].DataLength()

            print("%s%s %10s %s %s" %
                  (name,
                  " " * (max_name_len - len(name)),
                  self.format_centre(model.types[tensor.Type()],12),
                  self.format_left(shape_str,max(9, max_size_len)),
                  self.format_left(self.format_bytes(size_bytes),14)))
            
  def get_weights_size(self, model):
    size = 0
    weights_num = 0
    for weight_type in model.weight_types:
        totals = model.weight_types[weight_type]
        if totals['weights'] > 0:
          size += totals['bytes']
          weights_num += totals['weights']

    return weights_num, size

In [5]:
def read_model(file_name, GRAPH_INDEX = 0):
  tflite_file = None
  try:
      tflite_file = open(file_name, 'rb')
  except IOError:
      print("Failed to open file \"%s\"." % file_name)
      quit()

  print("=" * (len(file_name)+14+21))
  print("====== Reading flatbuffer \"%s\" ======" % file_name)
  print("=" * (len(file_name)+14+21))
  flatbuffer = tflite_file.read()
  print("Model Read Successful")

  base_name = file_name
  if base_name[-7:] == ".tflite":
      base_name = base_name[:-7]
          
  return reader.AnalysedTFliteModel(flatbuffer, GRAPH_INDEX)

In [6]:
file_name = 'model/car_sensor_reduced_drange.tflite'
model = read_model(file_name, GRAPH_INDEX = 0)

====== Reading flatbuffer "model/car_sensor_reduced_drange.tflite" ======
Model Read Successful


In [7]:
utils = Utils()
weights_num, size = utils.get_weights_size(model)
print(weights_num, size)

1809 2628


In [47]:
outdir = 'output'
if not os.path.isdir(outdir):
    os.mkdir(outdir)
output_report = outdir+'/report.csv'
cmd = 'tensorflow/bazel-bin/tensorflow/lite/tools/benchmark/benchmark_model --graph=model/car_sensor_reduced_drange.tflite --enable_op_profiling=true --profiling_output_csv_file="'+output_report+'"'
os.system(cmd)
#output = subprocess.check_output(['tensorflow/bazel-bin/tensorflow/lite/tools/benchmark/benchmark_model',
#                         '--graph='+file_name, '-enable_op_profiling=true', '--profiling_output_csv_file=' + 'report.csv']).decode('utf-8')
#init = int(re.search("\d+", re.search("Init: \d+",output).group(0)).group(0))
#inf_1st = int(re.search("\d+", re.search("First inference: \d+",output).group(0)).group(0))
#warm = int(re.search("\d+", re.search("Warmup (avg): \d+",output).group(0)).group(0))

0

In [49]:
with open(output_report, mode='r') as report:
    csv_reader = csv.reader(report, delimiter=',')
    memory_rows = [['Memory Summary:'],
              ['Number of Weights:', str(weights_num)],
              ['Weights Size (KB):', str(size)],
              ['=================================================================='],
              []]
    rows = []
    for row in memory_rows:
        rows.append(row)
    for row in csv_reader:
        rows.append(row)
        
with open(output_report, mode='w') as report:        
    csv_writer = csv.writer(report, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)

    for row in rows:    
        csv_writer.writerow(row)

[[['Memory Summary:'],
  ['Number of Weights:', '1809'],
  ['Weights Size (KB):', '2628'],
  ['==================================================================']],
 ["['Memory Summary:']",
  "['Number of Weights:', '1809']",
  "['Weights Size (KB):', '2628']",
  "['==================================================================']"],
 ['Profiling Info for Benchmark Initialization:'],
 ['============================== Run Order =============================='],
 ['node type',
  ' start',
  ' first',
  ' avg_ms',
  ' %',
  ' cdf%',
  ' mem KB',
  ' times called',
  ' name'],
 ['AllocateTensors',
  ' 0',
  ' 0.073',
  ' 0.073',
  ' 100%',
  ' 100%',
  ' 0',
  ' 1',
  ' AllocateTensors/0'],
 [],
 ['============================== Top by Computation Time =============================='],
 ['node type',
  ' start',
  ' first',
  ' avg_ms',
  ' %',
  ' cdf%',
  ' mem KB',
  ' times called',
  ' name'],
 ['AllocateTensors',
  ' 0',
  ' 0.073',
  ' 0.073',
  ' 100%',
  ' 100%',
  ' 0',
  ' 1